In [1]:

import os, time, json, pandas as pd, numpy as np
from datetime import datetime

def validate_pk(df, cols):
    if not all(c in df.columns for c in cols):
        return False, f"PK columns missing: {set(cols)-set(df.columns)}"
    dups = df.duplicated(subset=cols).sum()
    return dups == 0, f"Duplicate rows by PK: {dups}"

def validate_fk(df_from, col_from, df_to, col_to):
    if col_from not in df_from.columns or col_to not in df_to.columns:
        return False, f"Columns not found: {col_from} or {col_to}"
    missing = ~df_from[col_from].isin(df_to[col_to])
    return missing.sum() == 0, f"Unmatched rows: {missing.sum()}"

start = time.time()
print("Start: FINAL processing with automated ingestion and validations")

cwd = os.getcwd()
files = {
    "player": "player_final.csv",
    "team": "team_final.csv",
    "game": "game_final.csv",
    "summary": "game_summary_final.csv",
    "stats": "other_stats_final.csv"
}
dfs = {}
for k, fname in files.items():
    p = os.path.join(cwd, fname)
    if os.path.exists(p):
        dfs[k] = pd.read_csv(p, low_memory=False)
        print(f"Loaded {fname} with shape {dfs[k].shape}")
    else:
        print(f"Warning: {fname} not found")

player = dfs.get("player", pd.DataFrame())
team = dfs.get("team", pd.DataFrame())
game = dfs.get("game", pd.DataFrame())
summary = dfs.get("summary", pd.DataFrame())
stats = dfs.get("stats", pd.DataFrame())

# PK checks
if not team.empty:
    ok, msg = validate_pk(team, ["team_id"]); print(f"dim_team PK team_id: {ok}. {msg}")
if not player.empty:
    ok, msg = validate_pk(player, ["player_id"]); print(f"dim_player PK player_id: {ok}. {msg}")
if not game.empty:
    ok, msg = validate_pk(game, ["game_id"]); print(f"fact_game PK game_id: {ok}. {msg}")
if not stats.empty and all(c in stats.columns for c in ["game_id","player_id"]):
    ok, msg = validate_pk(stats, ["game_id","player_id"]); print(f"fact_player_game PK (game_id, player_id): {ok}. {msg}")

# FK checks
if not game.empty and not team.empty and "team_id" in team.columns:
    for col in ["home_team_id","visitor_team_id"]:
        if col in game.columns:
            ok, msg = validate_fk(game, col, team, "team_id")
            print(f"FK fact_game.{col} -> dim_team.team_id: {ok}. {msg}")
if not stats.empty and not game.empty:
    ok, msg = validate_fk(stats, "game_id", game, "game_id"); print(f"FK fact_player_game.game_id -> fact_game.game_id: {ok}. {msg}")
if not stats.empty and not player.empty:
    ok, msg = validate_fk(stats, "player_id", player, "player_id"); print(f"FK fact_player_game.player_id -> dim_player.player_id: {ok}. {msg}")

# Ingestion automation: detect new data against previous snapshot
status_file = os.path.join(cwd, "data_status.csv")
current = {k: (dfs[k].shape[0] if k in dfs and not dfs[k].empty else 0) for k in files}
prev = {}
if os.path.exists(status_file):
    try:
        prev_df = pd.read_csv(status_file)
        prev = dict(zip(prev_df["table"], prev_df["rows"]))
    except Exception:
        prev = {}

changes = {}
for k, n in current.items():
    delta = n - prev.get(k, 0)
    if delta != 0:
        changes[k] = delta

if changes:
    print("Ingestion changes detected since last run:")
    for k, d in changes.items():
        sign = "+" if d >= 0 else ""
        print(f"{k}: {sign}{d} rows vs previous run")
else:
    print("No ingestion changes detected since last run")

# Persist snapshot for next executions
pd.DataFrame({"table": list(current.keys()), "rows": list(current.values())}).to_csv(status_file, index=False, encoding="utf-8-sig")
print(f"Saved ingestion status snapshot to {status_file}")

# Generate SQL Server DDL
ddl_sqlserver = '''
-- NBA Star Schema for SQL Server
CREATE TABLE dim_team (
    team_id INT PRIMARY KEY,
    full_name NVARCHAR(100) NULL,
    abbreviation NVARCHAR(10) NULL,
    nickname NVARCHAR(50) NULL,
    city NVARCHAR(50) NULL,
    state NVARCHAR(50) NULL
);
CREATE TABLE dim_player (
    player_id INT PRIMARY KEY,
    full_name NVARCHAR(100) NULL,
    position NVARCHAR(20) NULL,
    height_cm FLOAT NULL,
    weight_kg FLOAT NULL,
    nationality NVARCHAR(50) NULL
);
CREATE TABLE fact_game (
    game_id INT PRIMARY KEY,
    game_date_est DATE NULL,
    season INT NULL,
    home_team_id INT NULL,
    visitor_team_id INT NULL,
    game_status_id INT NULL,
    FOREIGN KEY (home_team_id) REFERENCES dim_team(team_id),
    FOREIGN KEY (visitor_team_id) REFERENCES dim_team(team_id)
);
CREATE TABLE fact_player_game (
    game_id INT NOT NULL,
    player_id INT NOT NULL,
    team_id INT NULL,
    season INT NULL,
    points FLOAT NULL,
    assists FLOAT NULL,
    rebounds FLOAT NULL,
    minutes_played FLOAT NULL,
    PRIMARY KEY (game_id, player_id),
    FOREIGN KEY (game_id) REFERENCES fact_game(game_id),
    FOREIGN KEY (player_id) REFERENCES dim_player(player_id)
);
'''
ddl_path = os.path.join(cwd, "nba_schema_sqlserver.sql")
with open(ddl_path, "w", encoding="utf-8") as f:
    f.write(ddl_sqlserver)
print(f"Wrote SQL DDL to {ddl_path}")

end = time.time()
print(f"Completed FINAL processing in {end - start:.2f} seconds")

Start: FINAL processing with automated ingestion and validations
Loaded player_final.csv with shape (4831, 5)
Loaded team_final.csv with shape (30, 7)
Loaded game_final.csv with shape (65642, 55)
Loaded game_summary_final.csv with shape (58021, 14)
Loaded other_stats_final.csv with shape (28261, 26)
dim_team PK team_id: False. PK columns missing: {'team_id'}
dim_player PK player_id: False. PK columns missing: {'player_id'}
fact_game PK game_id: True. Duplicate rows by PK: 0
FK fact_player_game.game_id -> fact_game.game_id: True. Unmatched rows: 0
FK fact_player_game.player_id -> dim_player.player_id: False. Columns not found: player_id or player_id
Ingestion changes detected since last run:
stats: -10 rows vs previous run
Saved ingestion status snapshot to c:\Users\juanl\Downloads\final\csv\Daft17_Group01_PF\data\data_status.csv
Wrote SQL DDL to c:\Users\juanl\Downloads\final\csv\Daft17_Group01_PF\data\nba_schema_sqlserver.sql
Completed FINAL processing in 0.82 seconds
